In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as smp
from scipy.integrate import odeint
import plotly.graph_objects as go
from IPython.display import HTML
from sympy.matrices import Matrix
from sympy.vector import Vector
import plotly.express as px
from ipywidgets import interact
from ipywidgets import interactive
from ipywidgets import widgets
from scipy.integrate import odeint
import plotly.graph_objects as go
from IPython.display import HTML
from scipy.integrate import solve_ivp

### Free Energy Density
We start by the free energy density in terms of unit magnetizations $\textbf{m}$:

$$ \frac{\epsilon}{\hbar\gamma_{0}} =-\textbf{m}\cdot H-K_{z}(\textbf{e}_{h}\cdot \textbf{m})^2 $$

### LLG Equation
Dynamics are described by two coupled LLG equation:

$$\frac{d\vec{m}}{dt}= \gamma_{0} (\vec{\textbf{m}} \times H^{eff})+ \alpha_{0} \vec{\textbf{m}} \times\frac{d\vec{\textbf{m}}}{dt} +\gamma_{0}H_{DL}\vec{\textbf{m}}\times[\hat{y}\times \vec{\textbf{m}}] $$

### Effective Field
Where the effective fields for each subllattice $H^{eff}$ is given by:

$$H^{eff} =\frac{\delta\epsilon}{\hbar\gamma_{0}\delta\vec{\textbf{m}}} $$

We define the necessary variables for symbolic calculation using Sympy:

In [2]:
t = smp.symbols('t', real=True)
Hx, Hy, Hz, a, g, FL, DL, Hex, Kz,D= smp.symbols('Hx Hy Hz a g H_{FL} H_{DL} H_{ex} K_{z} D ', real=True)
mx, my, mz = smp.symbols(r'm_{x} m_{y} m_{z}', cls=smp.Function)
mx = mx(t)
my = my(t)
mz = mz(t)
# Derivatives
mx_d = smp.diff(mx,t)
my_d = smp.diff(my,t)
mz_d = smp.diff(mz,t)

m=Matrix([mx,my,mz])

m_d=Matrix([mx_d,my_d,mz_d])
H=Matrix([Hx,Hy,Hz])

X=Matrix([1,0,0])
Y=Matrix([0,1,0])
Z=Matrix([0,0,1])

Then we set the values of material parameters such as exchange, Gilbert damping, DMI and anisotropy:

In [3]:
a_value=0.1

Kz_value=0.01
g_value=1

DL_value=3E-5

Hx_value,Hy_value,Hz_value=1,1,1

e=Matrix([0,0,1])

Setting the free energy and calculating effective fields:

In [4]:
FE=  Kz*((e.dot(m))) - H.dot(m)
Heff=smp.diff(FE,m)


In [5]:
Heff

Matrix([
[        -Hx],
[        -Hy],
[-Hz + K_{z}]])

Now we define the 2-coupled LLG equations for each sublattice moment:

LLG Equation:

$$\frac{d\vec{m}}{dt}= \gamma_{0} (\vec{\textbf{m}} \times H^{eff})+ \alpha_{0} \vec{\textbf{m}} \times\frac{d\vec{\textbf{m}}}{dt} +\gamma_{0}H_{DL}\vec{\textbf{m}}\times[\hat{y}\times \vec{\textbf{m}}] $$

In [6]:
llg=-m_d + g*(m.cross(Heff))+a*(m.cross(m_d))+g*DL*m.cross(Y.cross(m))
llg=-m_d + g*(m.cross(Heff))+a*(m.cross(m_d))

We solve symbollically both equations for $\frac{d\vec{m}_{1,2}}{dt}$: (we don't solve the differential equation yet. We just have a symbollic expression for $\frac{d\vec{m}_{1,2}}{dt}$)

In [7]:
sols= smp.solve(llg, (mx_d, my_d, mz_d),simplify=True, rational=False)

We evaluate the symbolic expressions:

In [8]:
mx_df = smp.lambdify((t,m), sols[mx_d].subs([(g,g_value),(a,a_value),(Kz,Kz_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))
my_df = smp.lambdify((t,m), sols[my_d].subs([(g,g_value),(a,a_value),(Kz,Kz_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))
mz_df = smp.lambdify((t,m), sols[mz_d].subs([(g,g_value),(a,a_value),(Kz,Kz_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))

We define a function and inital conditions so that we can solve the ODES numerically:

In [9]:
mx_df(1,[0,0,1])

1.0891089108910892

In [10]:
def dSdt(t,S):
    mx,my,mz  = S
    return [
        mx_df(t,[mx,my,mz]),
        my_df(t,[mx,my,mz]),
        mz_df(t,[mx,my,mz]),
    ]

theta=30*(np.pi/180)
phi=45*(np.pi/180)

x=np.sin(theta)*np.cos(phi)
y=np.sin(theta)*np.sin(phi)
z=np.cos(theta)



init_cond=(x,y,z)

We solve the coupled ODEs using ODEINT function:

In [11]:
t = np.linspace(0, 100, 500)
number_of_points=len(t)
sol = odeint(dSdt, y0=init_cond, t=t, tfirst=True)


In [12]:
sol=sol.T

We plot the graphics:

In [13]:
X1s=sol[0]
Y1s=sol[1]
Z1s=sol[2]

fig1 = go.FigureWidget()
#NEEL VECTOR1
x1 = np.array([0,X1s[0]])
y1 = np.array([0,Y1s[0]])
z1 = np.array([0,Z1s[0]])

fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.75,
    marker=dict(size=0.1,color='black'),
    line=dict(color='red', width=4 ),
    name="$m_{1}$")
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
    sizeref=0.1,colorscale=[[0, 'red'], [1,'red']])





#Damping-like Torque
fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.9,
    marker=dict(size=0.1,color='black'),
    line=dict(color='brown', width=5 ),
    name="$ \\tau_{DL}$",showlegend=True)

fig1.update_layout(
    height=700,
    width=900,

    autosize=True,
    scene=dict(camera=dict(up=dict(x=0,y=0, z=1 ), eye=dict(x=-1,y=1,z=1,)),
               xaxis = dict(nticks=6, range=[-2,2]),
               yaxis = dict(nticks=6, range=[-2,2]),
               zaxis = dict(nticks=6, range=[-2,2]),
        aspectratio = dict( x=1, y=1, z=1),
        aspectmode = 'manual'))
#CIRCLE IN XY PLANE
C1=1
phi=np.linspace(0,2*np.pi,100)
x=C1*np.cos(phi)
y=C1*np.sin(phi)
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)
#CIRCLE IN XZ PLANE
phi=np.linspace(0,2*np.pi,100)
x=C1*np.cos(phi)
y=C1*np.zeros(len(x))
z=C1*np.sin(phi)
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

#CIRCLE IN YZ PLANE
phi=np.linspace(0,2*np.pi,100)
x=C1*np.zeros(len(x))
y=C1*np.cos(phi)
z=C1*np.sin(phi)
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

#CIRCLE IN YZ PLANE
phi=np.linspace(-1,1,100)
x=C1*np.zeros(len(x))
y=C1*np.zeros(len(x))
z=phi
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)
#CIRCLE IN YZ PLANE
phi=np.linspace(-1,1,100)
x=C1*np.zeros(len(x))
y=phi
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)
#CIRCLE IN YZ PLANE
phi=np.linspace(-1,1,100)
x=phi
y=C1*np.zeros(len(x))
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)


theta, phi = np.mgrid[0:2*np.pi:100j, 0:2*np.pi:100j]
x = np.sin(theta)*np.sin(phi)
y = np.sin(theta)*np.cos(phi)
z = np.cos(theta)

fig1.add_surface(x=x,y=y,z=z,opacity=0.07,colorscale=[[0, 'grey'], [1,'white']],showlegend=False, showscale = False)

fig1.add_scatter3d(x=X1s, y=Y1s, z=Z1s,opacity=0.25,
    marker=dict(size=0.1,color='red'),
    line=dict(color='red', width=3),
    showlegend=False)

#Extermal Field1
norm=2*np.sqrt(float(Hx_value**2+Hy_value**2+Hz_value**2))
x1 = np.array([0,Hx_value/norm])
y1 = np.array([0,Hy_value/norm])
z1 = np.array([0,Hz_value/norm])

fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.75,
    marker=dict(size=0.1,color='black'),
    line=dict(color='green', width=6 ),
    name="$H_{applied}$")
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
    sizeref=0.1,colorscale=[[0, 'green'], [1,'green']])

#Spin Accumulation

x1 = np.array([0,0])
y1 = np.array([0,0.5])
z1 = np.array([0,0])

fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.75,
    marker=dict(size=0.1,color='black'),
    line=dict(color='yellow', width=6),
    name="$\sigma_{AC}$")
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
    sizeref=0.1,colorscale=[[0, 'yellow'], [1,'yellow']])


def update(t=0):
    with fig1.batch_update():
        
        fig1.data[0].x=[0,X1s[t]]
        fig1.data[0].y=[0,Y1s[t]]
        fig1.data[0].z=[0,Z1s[t]]
        
        fig1.data[1].x=[X1s[t]]
        fig1.data[1].y=[Y1s[t]]
        fig1.data[1].z=[Z1s[t]]
        fig1.data[1].u=[2*X1s[t]]
        fig1.data[1].v=[2*Y1s[t]]
        fig1.data[1].w=[2*Z1s[t]]
        


w=interact(update,t=widgets.IntSlider(min=0, max=number_of_points-1, step=int(number_of_points)/25, value=0,description="Time Step",readout_format=".1d",continuous_update=False))
#w=interactive(update,t=(0,number_of_points-1,1)) 
#w.children[0].description="$Time (t)$"
#display(w)
fig1



interactive(children=(IntSlider(value=0, continuous_update=False, description='Time Step', max=499, readout_fo…

FigureWidget({
    'data': [{'line': {'color': 'red', 'width': 4},
              'marker': {'color': 'black', …

In [14]:
# Import data
import time
import numpy as np
length=len(X1s)
step=int(length/30)
step=5
# Define frames
import plotly.graph_objects as go

fig = go.Figure(frames=[go.Frame(data=(go.Scatter3d(x=[0,X1s[k]],y=[0,Y1s[k]],z=[0,Z1s[k]],marker=dict(size=0.1,color='black'),
                                    line=dict(color='red', width=4)),
                                       
                                       
                       go.Scatter3d(x=[0,X2s[k]],y=[0,Y2s[k]],z=[0,Z2s[k]],marker=dict(size=0.1,color='black'),
                                    line=dict(color='blue', width=4)),
                       go.Cone(x=[X1s[k]], y=[Y1s[k]], z=[Z1s[k]], u=[X1s[k]+X1s[k]], v=[Y1s[k]+Y1s[k]], w=[Z1s[k]+Z1s[k]],
                               opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",sizeref=0.1,
                               colorscale=[[0, 'red'], [1,'red']]),
                       go.Cone(x=[X2s[k]], y=[Y2s[k]], z=[Z2s[k]], u=[X2s[k]+X2s[k]], v=[Y2s[k]+Y2s[k]], w=[Z2s[k]+Z2s[k]],
                               opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",sizeref=0.1,
                               colorscale=[[0, 'blue'], [1,'blue']])                      
                      ),  
    name=str(k))
    for k in np.arange(0,length,step)]
               )

k=0
# Add data to be displayed before animation starts
fig.add_trace(go.Scatter3d(
    x=[0,X1s[k]],y=[0,Y1s[k]],z=[0,Z1s[k]],marker=dict(size=0.1,color='black'),
                                    line=dict(color='red', width=4)))
fig.add_trace(go.Scatter3d(
    x=[0,X2s[k]],y=[0,Y2s[k]],z=[0,Z2s[k]],marker=dict(size=0.1,color='black'),
                                    line=dict(color='blue', width=4)))
fig.add_trace(go.Cone(x=[X1s[k]], y=[Y1s[k]], z=[Z1s[k]], u=[X1s[k]+X1s[k]], v=[Y1s[k]+Y1s[k]], w=[Z1s[k]+Z1s[k]],
                      opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",sizeref=0.1,
                      colorscale=[[0, 'red'], [1,'red']]))
              
fig.add_trace(go.Cone(x=[X2s[k]], y=[Y2s[k]], z=[Z2s[k]], u=[X2s[k]+X2s[k]], v=[Y2s[k]+Y2s[k]], w=[Z2s[k]+Z2s[k]],
                      opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",sizeref=0.1,
                      colorscale=[[0, 'blue'], [1,'blue']]))

fig.add_trace(go.Scatter3d(x=X1s, y=Y1s, z=Z1s,opacity=0.25,
    marker=dict(size=0.1,color='red'),
    line=dict(color='red', width=3.5),
    showlegend=False))

fig.add_trace(go.Scatter3d(x=X2s, y=Y2s, z=Z2s,opacity=0.25,
    marker=dict(size=0.1,color='blue'),
    line=dict(color='blue', width=3.5 ),
    showlegend=False))


#CIRCLE IN XY PLANE
C1=1
phi=np.linspace(0,2*np.pi,60)
x=C1*np.cos(phi)
y=C1*np.sin(phi)
z=C1*np.zeros(len(x))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False))

#CIRCLE IN XZ PLANE
phi=np.linspace(0,2*np.pi,60)
x=C1*np.cos(phi)
y=C1*np.zeros(len(x))
z=C1*np.sin(phi)
fig.add_trace(go.Scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False))

#CIRCLE IN YZ PLANE
phi=np.linspace(0,2*np.pi,60)
x=C1*np.zeros(len(x))
y=C1*np.cos(phi)
z=C1*np.sin(phi)
fig.add_trace(go.Scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False))

#Line at Z axis
phi=np.linspace(-1,1,2)
x=C1*np.zeros(len(x))
y=C1*np.zeros(len(x))
z=phi
fig.add_trace(go.Scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False))
#Line at Y axis
phi=np.linspace(-1,1,2)
x=C1*np.zeros(len(x))
y=phi
z=C1*np.zeros(len(x))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False))
#Line at X axis
phi=np.linspace(-1,1,2)
x=phi
y=C1*np.zeros(len(x))
z=C1*np.zeros(len(x))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False))

theta, phi = np.mgrid[0:2*np.pi:100j, 0:2*np.pi:100j]
x = np.sin(theta)*np.sin(phi)
y = np.sin(theta)*np.cos(phi)
z = np.cos(theta)
fig.add_trace(go.Surface(x=x,y=y,z=z,opacity=0.07,colorscale=[[0, 'grey'], [1,'white']],showlegend=False, showscale = False))    

def frame_args(duration):
    return {
            "frame": {"duration": duration,"redraw": True},
            "mode": "immediate",
            "fromcurrent": False,
            "transition": {"duration": duration, "easing":"linear"}
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout

fig.update_layout(
         title='AFM Neel Vector Dynamics',
         width=900,
         height=700,
         scene=dict(camera=dict(up=dict(x=0,y=0, z=1 ), eye=dict(x=-0.9,y=0.9,z=0.9,)),
                xaxis = dict(nticks=6, range=[-1.5,1.5]),
                yaxis = dict(nticks=6, range=[-1.5,1.5]),
                zaxis = dict(nticks=6, range=[-1.5,1.5]),
         aspectratio = dict( x=1, y=1, z=1),
         aspectmode = 'manual'),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(1)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                        
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()

NameError: name 'X2s' is not defined